In [47]:
import pandas as pd
import numpy as np
import sys
import os

# Allows to get the module in utils
sys.path.append(os.path.abspath(".."))


from utils.load import load_data
from utils.preprocessing import preprocess
from utils.split import split_data

# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
# from sklearn.compose import ColumnTransformer

import tensorflow as tf


In [48]:
df = load_data()
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,female,0,yes,no,1,no,no phone service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic check,29.85,29.85,no
1,5575-GNVDE,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one year,no,mailed check,56.95,1889.50,no
2,3668-QPYBK,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed check,53.85,108.15,yes
3,7795-CFOCW,male,0,no,no,45,no,no phone service,dsl,yes,...,yes,yes,no,no,one year,no,bank transfer (automatic),42.30,1840.75,no
4,9237-HQITU,female,0,no,no,2,yes,no,fiber optic,no,...,no,no,no,no,month-to-month,yes,electronic check,70.70,151.65,yes


In [49]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(df)
# Check the data
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((4507, 20), (1127, 20), (1409, 20), (4507,), (1127,), (1409,))

In [50]:
X_train_processed, X_test_processed, X_val_processed, y_test_encoding, y_train_encoded, y_val_encoding, pipeline, le = preprocess(X_train, X_val, X_test, y_train, y_val, y_test)

In [28]:
# Benjamin model

def build_model(input_dim, num_classes=1):
    # Réseau avec 2 couches cachées de 64 neurones chacune
    # et une couche de sortie avec activation softmax pour classification
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)), #4 car 4 features
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='sigmoid') #Remplassement num_classes de son code par le nombre de cat différente dans y
    ])

    # Définition de la fonction de perte, de l'optimiseur et des métriques
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), #Adam=fonction coût. C'est une descente de gradient amélioré mais pas maîtrisé par le formateur
        loss='binary_crossentropy' if num_classes == 1 else 'categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    return model

In [38]:
input_dim = X_train_processed.shape[1]
model = build_model(input_dim=input_dim, num_classes=1)

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_23 (Dense)                │ (None, 64)             │       290,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 294,593 (1.12 MB)

 Trainable params: 294,593 (1.12 MB)

 Non-trainable params: 0 (0.00 B)

In [39]:
history = model.fit(
    X_train_processed, y_train_encoded,
    validation_data=(X_val_processed, y_val_encoding),
    epochs=20,
    batch_size=16,
    verbose=1
)

Epoch 1/20
262/282 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7349 - loss: nan

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_23" is incompatible with the layer: expected axis -1 of input shape to have value 4536, but received input with shape (None, 1156)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 1156), dtype=float32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>

In [36]:
y_train_encoded

array([0, 0, 0, ..., 0, 0, 0], shape=(4507,))

In [43]:
# Vérifier les NaN ou inf
X_train_dense=X_train_processed.toarray()
print(np.any(np.isnan(X_train_dense)))  # Doit être False
# print(np.any(np.isinf(X_train_processed)))  # Doit être False


True


In [41]:
X_train_processed

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 60935 stored elements and shape (4507, 4536)>

In [44]:
X_train_dense

array([[-0.43927126, -0.2553918 , -0.17476006, ...,  1.        ,
         0.        ,  0.        ],
       [ 2.27649768,  1.61043169,  1.71099346, ...,  0.        ,
         0.        ,  0.        ],
       [-0.43927126,  1.52930893,  1.65268836, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-0.43927126, -1.26942631, -0.67118634, ...,  1.        ,
         0.        ,  0.        ],
       [-0.43927126, -0.57988284, -0.4896076 , ...,  0.        ,
         0.        ,  1.        ],
       [-0.43927126,  0.39359028,  0.98967608, ...,  0.        ,
         1.        ,  0.        ]], shape=(4507, 4536))

In [51]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64